Business Context:

As a business leader at Orion Tech Solutions, you (“Alex Carter”) oversee multiple software development and IT infrastructure projects. Your responsibilities include coordinating with stakeholders, managing escalations, and ensuring timely deliveries. With hundreds of emails flooding your inbox daily, manually sorting through them is time-consuming and increases the risk of missing critical updates, client escalations, or project approvals.

Objective:
The goal of this project is to develop a Generative AI-powered system that:
✅ Summarizes emails into actionable insights using the Yesterbox approach (excluding today’s emails).
✅ Prioritizes emails based on urgency, sender, and context.
✅ Draft context-aware responses to reduce manual effort.
✅ Evaluate the drafted context-aware responses using LLM-as-a-Judge.
Tasks & Workflow
Task 1: Generate a Detailed Summary of Yesterday’s Inbox
Task 1A: Executive Dashboard (Top-Level Summary of Yesterday’s Emails)
Sample Output:
🔹 Total Emails from Yesterday: 100
🔹 🛑 Urgent & High-Priority Emails: 10 (Require Immediate Action Today)
🔹 ⚡ Deadline-Driven Emails: 8 (Must Be Addressed Today)
🔹 📌 Routine Updates & Check-ins: 35 (Review & Acknowledge)
🔹 📎 Non-Urgent & Informational Emails: 45 (Can Be Deferred or Delegated)
🔹 🎉 Personal & Social Emails: 22 (Optional Review)
🔹 🗑️ Spam/Unimportant Emails Filtered Out: 20

AI Conclusion:
"You have 18 critical emails from yesterday that require action today. Additionally, there are 35 updates to review at your convenience."

Task 1B: Analyze Urgent & High-Priority Emails (🛑 Must-Do First Today)
Focus on emails that require immediate action and impact critical projects or client relationships.

Task 1C: Review Deadline-Driven Emails (⚡ Needs Attention Today)
Identify emails tied to important deadlines and ensure timely responses.

Task 2: AI-Generated Response Drafts for Critical Email
For each Urgent & High-priority email or Deadline-Driven email from yesterday, generate an AI-powered response draft for quick review and editing before sending.

NOTE : Critical Emailsare the combination of Urgent & High-Priority Emails + Deadline-Driven Emails

Task 3: Validate AI-Generated Results Using the "LLM as a Judge" Technique
To ensure accuracy and reliability, apply the "LLM as a Judge" technique to evaluate:
✅ Relevance: How well does thesummaryaddress the input query or task?
✅ Clarity: How clear and understandable is thesummary?
✅ Actionability: Does thesummaryprovide clear next steps or actionable information?
✅ Strengths: Highlight the key strengths of the summary.
✅ Improvements: Suggest 1-2 areas for improvement.
✅ Overall Justification: Provide a 2-3 line summary evaluation, including key observations.

In [15]:
# Parse JSON content from files
import json
# Use pathlib for convenient path operations
from pathlib import Path

# Always load API_KEY from the local `config.json` file; ignore environment variables
cfg_path = Path('/content/JHU_Learnings/Week 9 Project/config.json')
API_KEY = None
# Check that the config file exists before attempting to read it
if cfg_path.exists():
    try:
        # Open and parse the config file as JSON
        with open(cfg_path, 'r', encoding='utf-8') as f:
            cfg = json.load(f)
        # Look for common key names and take the first non-empty value
        API_KEY = cfg.get('API_KEY') or cfg.get('api_key') or cfg.get('openai_api_key')
    except Exception:
        # If reading or parsing fails, leave API_KEY as None
        API_KEY = None
else:
    # Inform the user if the file is missing
    print(f'{cfg_path} not found.')

# Provide a short confirmation but do not print the secret itself
if API_KEY:
    print(f'API_KEY loaded from {cfg_path} (length {len(API_KEY)})')
else:
    print(f'API_KEY not found in {cfg_path}. Add it to the file.')

API_KEY loaded from /content/JHU_Learnings/Week 9 Project/config.json (length 67)


In [17]:
# Print a masked form of the API key so the full secret is not exposed
# If API_KEY is set, show first 4 and last 4 characters separated by '...'; otherwise print None
api_key_val = globals().get('API_KEY')
if api_key_val:
    print(api_key_val[:8] + '...' + api_key_val[-8:])
else:
    print(api_key_val)

gl-U2Fsd...oxDCQTE2


In [19]:
!pip install -q openai==1.61.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.1/463.1 kB 21.6 MB/s eta 0:00:00


In [20]:
import openai
print(openai.__version__)

1.61.1


In [22]:
# @title Loading the `config.json` file
import json, os

# Load the JSON file and extract values
file_name = '/content/JHU_Learnings/Week 9 Project/config.json'
with open(file_name, 'r') as file:
    config = json.load(file)
    os.environ['OPENAI_API_KEY']  = config.get("API_KEY") # Loading the API Key
    os.environ["OPENAI_BASE_URL"] = config.get("OPENAI_API_BASE") # Loading the API Base Url

In [23]:
model_name = "gpt-4o-mini"

In [25]:
from openai import OpenAI

# Initialize OpenAI client
client = OpenAI()

In [26]:
# @title LLM function
# @markdown Once the API details are filled, the notebook will automatically load the configuration, and learners can generate model outputs using the llm() function.


def llm(system_prompt, user_prompt):
  try:
      # Craft the messages to pass to chat.completions.create
      prompt = [
          {'role':'system', 'content': system_prompt},
          {'role': 'user', 'content': user_prompt}
      ]

      response = client.chat.completions.create(
          model=model_name,
          messages=prompt,
          temperature=0
      )

      return response.choices[0].message.content.strip()

  except Exception as e:
      prediction = f'Sorry, I encountered the following error: \n {e}'
      print(prediction)

In [ ]:
# @title Step 1: Load the Dataset
# Data Loading

import pandas as pd
# Use the raw GitHub URL for the CSV file
#df = pd.read_csv("https://raw.githubusercontent.com/linufx2208-sketch/JHU_Learnings/main/Week%209%20Project/Alex_emails_march_04.csv", index_col="email_id", encoding='latin-1')      #Add the data file location
df = pd.read_csv("/content/JHU_Learnings/Week 9 Project/Alex_emails_march_04.csv", index_col="email_id", encoding='latin-1')
df

In [29]:
# @title Step 2: Apply Yesterbox Filtering
# @markdown The Yesterbox approach involves processing emails from the previous day first before tackling today's emails.

# @markdown For this dataset, consider today's date as 4th March 2025.

# @markdown We filter the dataset to only include emails received on 3rd March 2025 (yesterday)
# (Yesterbox Approach)(Today: 4 march)



from datetime import datetime, timedelta

yesterday_date = pd.to_datetime("3/3/2025").strftime('%m/%d/%Y')

df['date_received'] = pd.to_datetime(df['date_received']).dt.strftime('%m/%d/%Y')

yesterday_emails = df[df['date_received'] == yesterday_date].reset_index(drop=True)
print(f"Filtered Emails Count: {len(yesterday_emails)}")


Filtered Emails Count: 51


In [30]:
df.shape

(60, 5)

In [31]:
# Here we see only 51 emails, as in 10 email had the date of 4th March 2025
yesterday_emails.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date_received   51 non-null     object
 1   sender          51 non-null     object
 2   subject         51 non-null     object
 3   body            51 non-null     object
 4   main_recipient  51 non-null     object
dtypes: object(5)
memory usage: 2.1+ KB


## **TASK - Categorization of emails**
Your task is to write the `system_prompt` & `user_prompt` for classifying all the emails into one of the below pre-defined categories.

1. **Urgent & High-Priority Emails**:
   - Emails that require immediate action and must be addressed today.
2. **Deadline-Driven Emails**:
   - Time-sensitive emails or meeting requests that need attention today.
3. **Routine Updates & Check-ins**:
   - Emails that require review and acknowledgment without immediate action.
4. **Non-Urgent Informational Emails**:
   - Emails that can be deferred or delegated to another time or person.
5. **Personal & Social Emails**:
   - Emails that can be reviewed optionally at a later time.
6. **Spam/Unimportant Emails**:
   - Emails that are not relevant and should be filtered out.


**Note:** Your response should include only one of the above six specified categories and nothing else.

```
Example Email : 'You Won a Free iPhone 15 Pro! ?? Click to Claim'
Expected Response : 'Spam/Unimportant Emails'

```




In [32]:
# @title Code to add categories to the dataset
categories = ['Urgent & High-Priority Emails',
              'Deadline-Driven Emails',
              'Routine Updates & Check-ins',
              'Non-Urgent Informational Emails',
              'Personal & Social Emails',
              'Spam/Unimportant Emails']

In [33]:
system_prompt = """
You are an enterprise email assistant responsible for classifying the importance of emails received yesterday.

Your task:
- Classify EACH email into EXACTLY ONE category from the list provided by the user.
- Output ONLY the category label. No explanations, no extra text, no formatting.

Core rules (STRICT):
1. Choose ONE category only.
2. If multiple categories seem applicable, choose the MORE URGENT one.
3. If the email signals risk to production, security, compliance, patient safety, or customer impact, treat it as HIGH priority even if no explicit deadline is stated.
4. If the email appears promotional, unsolicited, or phishing-like, classify it as Spam/Unimportant Emails.

CRITICAL URGENCY RULE (based on real enterprise incidents):
Classify as 'Urgent & High-Priority Emails' if the email mentions ANY of the following:
- Production outage, downtime, system crash, performance degradation
- Security vulnerabilities, exposed keys, authentication failures, CVEs, OWASP risks
- Compliance risk, audit risk, SLA breach, patient safety, healthcare data issues
- Missing data, data corruption, failed syncs impacting customers
- Escalation language (URGENT, ASAP, immediate attention, war room, escalation)

Deadline rule:
- If the email requires approval, action, or response by a specific date/time (EOD, COB, March X, SLA, before expiry), classify as 'Deadline-Driven Emails' unless it is already Urgent.

Routine rule:
- Status updates, daily/weekly progress reports, check-ins, and non-blocking coordination are 'Routine Updates & Check-ins'.

Informational rule:
- Industry trends, statistics, webinars, thought leadership, awareness emails with no required action are 'Non-Urgent Informational Emails'.

Personal rule:
- Greetings, confirmations, thank-you notes, social or non-work communication are 'Personal & Social Emails'.

Spam rule:
- Marketing emails, promotions, free trials, giveaways, unsolicited offers, suspicious links, prize claims are 'Spam/Unimportant Emails'.

Return ONLY the category label.
"""

In [34]:
user_prompt = f"""
Classify the following email into EXACTLY ONE of the categories below.
Return ONLY the category label exactly as written.

Categories:
- {categories[0]}
- {categories[1]}
- {categories[2]}
- {categories[3]}
- {categories[4]}
- {categories[5]}

Classification guidance:
- Production, security, compliance, healthcare, or customer-impacting risk → {categories[0]}
- Explicit deadline or approval date → {categories[1]}
- Status updates, progress, routine coordination → {categories[2]}
- Informational or awareness-only content → {categories[3]}
- Social or personal communication → {categories[4]}
- Promotions, scams, irrelevant marketing → {categories[5]}

Do not explain your answer.
"""

In [35]:
# @title Categorizing the emails
from tqdm import tqdm  # Import the tqdm library for the progress bar

# Ensure the 'category' column exists in the DataFrame
if 'category' not in yesterday_emails.columns:
  yesterday_emails['category'] = None  # Create the column if it does not exist


for index, row in tqdm(yesterday_emails.iterrows(), total=df.shape[0], desc='Processing emails'):
  prompt = f"""{user_prompt}
    Please find the attached email below from yesterday that need to be analyzed:
    ```
    {row.to_string()}
    ```
  """

  category_by_llm = llm(system_prompt, prompt)

  # Append the category generated by the LLM to the 'category' column in the same row
  if category_by_llm in categories:
    yesterday_emails.at[index, 'category'] = category_by_llm
  else:
    yesterday_emails.at[index, 'category'] = ""

Processing emails:  85%|████████▌ | 51/60 [00:56<00:09,  1.11s/it]


In [36]:
# Display the DataFrame which has categories
yesterday_emails.head()

,date_received,sender,subject,body,main_recipient,category
0,03/03/2025,Julia Martin,Approval Request: Budget Approval Needed by EOD,"Hi Alex,\n\nI hope you're doing well. As we ap...",Alex,Deadline-Driven Emails
1,03/03/2025,Fiona White,Are Your APIs Secure? Reddit & Discord Sound t...,"Hi Alex,\n\nA heated Discord discussion in the...",Alex,Non-Urgent Informational Emails
2,03/03/2025,Samantha Lee,Approval Needed: Project Scope Adjustment for ...,"Hi Alex,\n\nWeve encountered an unexpected AP...",Alex,Deadline-Driven Emails
3,03/03/2025,James Patel,Subject: Daily Update  Project Titan (March 3),"Hey Alex,\n\nQuick update on Project Titan for...",Alex,Routine Updates & Check-ins
4,03/03/2025,David Whitmore,[URGENT] Dashboard Syncing Issues  Production...,"Hey Alex,\n\nWeve got a big issue right nowl...",Alex,Urgent & High-Priority Emails
